# More complex configurations
## Ways to handle multiple clusters

Let's start from imports

In [ ]:
from __future__ import print_function
import time
from jobqueue_features.decorators import on_cluster, task
from jobqueue_features.functions import set_default_cluster
from jobqueue_features.clusters import CustomSLURMCluster
from jobqueue_features.clusters_controller import (
    clusters_controller_singleton as controller,
)

Then define default cluster as `CustomSLURMCluster`, because those example working on SLURM docker.

In [ ]:
set_default_cluster(
    CustomSLURMCluster
)

Let's default tasks to provide on our cluster.

In [ ]:
@on_cluster()
@task()
def square(x):
    return x ** 2


@on_cluster(
    cluster=CustomSLURMCluster(
        name="other", 
        walltime="00:01:00"
    )
)
@task(cluster_id="other")
def inc(x):
    return x + 1

Now we have two clusters, a default one and one with the name `other`. 

In [ ]:
def main():
    from distributed import as_completed
    
    sq_tasks = list(map(square, range(1, 11)))
    inc_tasks = list(map(inc, range(1, 11)))
    print(
        "Squares: ",
        [t.result() for t in as_completed(sq_tasks)]
    )
    print(
        "Increments: ",
        [t.result() for t in as_completed(inc_tasks)]
    )

Run it!

In [ ]:
start = time.time()
main()
print("Executed in {}".format(time.time() - start))

Let's clean up after ourselves

In [ ]:
controller._close()

## Configuration options

We've hidden away the configuration of cluster, so you've never really had to think about it. Unfortunately the configuration can be quite complicated depending on how your local system is configured.

First, let's look at the options that our `CustomSLURMCluster` can take (use `shift+tab` between the brackets below):

In [ ]:
CustomSLURMCluster()

Having to set all those options all the time is very tedious so instead we can use a Dask configuration file (see the [Dask documentation](https://docs.dask.org/en/latest/configuration.html) for details on where this can be placed). 

Luckily the system is the same for all users, so such a file can be created universally. Let's take a look at the configuration file for a complex system:

In [ ]:
%load /jobqueue_features/jobqueue_features/jobqueue_features.yaml

With this (admittedly complex) configuration file, it is relatively easy to configure different clusters to run on different parts of the resource (CPU, GPU, KNL) and have them communicate with each other.